In [12]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

ds = load_dataset("rajpurkar/squad")

In [13]:
model = AutoModelForCausalLM.from_pretrained("../distillLLAMA2")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [14]:
size=10
batch=1
train_dataset=ds["train"].shuffle(seed=42).select(range(size))


In [23]:
import torch
train_data=[]
for i in range(size):
    tokenized = tokenizer(train_dataset[i]['context']+train_dataset[i]['question'] + train_dataset[i]['answers']['text'][0], padding="max_length", max_length=512, truncation=True, return_tensors="pt")
    cqlen = len(tokenizer(train_dataset[i]['context']+train_dataset[i]['question'])['input_ids'])
    length=torch.sum(tokenized['attention_mask'][0])
    input_ids = tokenized['input_ids'].squeeze().tolist()
    attention_mask = tokenized['attention_mask'].squeeze().tolist()
    labels = input_ids[1:] + [tokenizer.pad_token_id]
    for i in range(len(attention_mask)):
        if attention_mask[i]==0:
            labels[i]=-100

    # labels=[-100]*512
    # for i in range(length-cqlen):
    #     labels[cqlen+i-1]=input_ids[cqlen+i]
    train_data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})

sbatch = int(size/batch)


input_ids = [item["input_ids"] for item in train_data]
labels = [item["labels"] for item in train_data]
attention_mask = [item["attention_mask"] for item in train_data]


In [24]:
from torch.optim import AdamW
device='cuda'
input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
labels_tensor = torch.tensor(labels, dtype=torch.long)
attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)
criterion.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
input_ids_tensor=input_ids_tensor.to(device)
labels_tensor=labels_tensor.to(device)
attention_mask_tensor = attention_mask_tensor.to(device)
model.to(device)
eval_loss=0
model.train()

eval_losses = []
vocab_size = model.config.vocab_size


In [25]:
input_ids=input_ids_tensor[0].unsqueeze(0)
labels=labels_tensor[0].unsqueeze(0)
attention_mask=attention_mask_tensor[0].unsqueeze(0)
optimizer.zero_grad()
outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
logits=outputs.logits
loss = criterion(logits.view(-1, vocab_size), labels.view(-1))

In [26]:
loss

tensor(12.3120, device='cuda:0', grad_fn=<NllLossBackward0>)